<a href="https://colab.research.google.com/github/hihunjin/Code-snippet-for-everything/blob/main/audio/wav2vec_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers
!pip install torchsummary

In [2]:
import os

import torch
import torch.nn as nn
from torchaudio.models.wav2vec2.utils import import_huggingface_model
from transformers import Wav2Vec2ForCTC
from torchaudio.models import wav2vec2_base
from torchsummary import summary

pt_loc = 'wav2vec2-base-960h.pt'
print(os.path.exists(pt_loc))
if not os.path.exists(pt_loc):
    original = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
    model_ = import_huggingface_model(original)
    torch.save(model_.state_dict(), pt_loc)

# Session 2 - Load model and the parameters
model = wav2vec2_base(num_out=32)
model.load_state_dict(torch.load(pt_loc))
model.encoder.readout = nn.Identity(768)
model.eval();

False


Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# https://github.com/pytorch/audio/blob/e885204ea0d343b8bddd7e829b9e7b29e7c5a9bb/torchaudio/models/wav2vec2/model.py#L10
# https://pytorch.org/audio/stable/models.html#wav2vec2model
    # def forward(
    #         self,
    #         waveforms: Tensor,
    #         lengths: Optional[Tensor] = None,
    # ) -> Tuple[Tensor, Optional[Tensor]]:
    #     """Compute the sequence of probability distribution over labels.
    #     Args:
    #         waveforms (Tensor): Audio tensor of shape `(batch, frames)`.
    #         lengths (Tensor or None, optional):
    #             Indicates the valid length of each audio sample in the batch.
    #             Shape: `(batch, )`.
    #     Returns:
    #         Tensor and an optional Tensor:
    #         Tensor
    #             The sequences of probability distribution (in logit) over labels.
    #             Shape: `(batch, frames, num labels)`.
    #         Tensor or None
    #             If ``lengths`` argument was provided, a Tensor of shape `(batch, )`
    #             is retuned. It indicates the valid length of each feature in the batch.
    #     """

ex = torch.rand((3,30000))
out = model(waveforms=ex)
print(out[0].size(), out[1])

(torch.Size([3, 93, 768]), None)